<a href="https://colab.research.google.com/github/FSALVA157/Awesome-Profile-README-templates/blob/master/RAG_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Construir una aplicación RAG desde cero**

Vamos a comenzar cargando las variables de entorno que necesitamos utilizar.

In [ ]:
!pip install python-dotenv
!pip install langchain-openai
!pip install langchain-community
!pip install pypdf
!pip install langchain
!pip install docarray
!pip install langchain_pinecone

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(OPENAI_API_KEY)
print(PINECONE_API_KEY)


sk-proj-MTaTUA3Sn5aHuXX7kW1xT3BlbkFJHc5d3oDJkbJFlY7kiZQC
6ce2990a-7aaa-455c-8ea7-0a8470afb8d0


## **Configuración del modelo**

Vamos a establecer el modelo de LLM que utilizaremos en nuestro flujo de trabajo.

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

Probamos que la configuración funciona preguntado algo al modelo



In [ ]:
model.invoke("2+2")

AIMessage(content='2+2 = 4', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 10, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2661e3b-f523-47d6-95c7-3558d6844887-0')

Vamos a parsear la respuesta para solo obtener el valor de content='2 + 2 equals 4.'

2 + 2 equals 4.


In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("2+2")

'4'

## **Prompt Template**

Las plantillas de prompts son recetas predefinidas para generar prompts para modelos de lenguaje.

Una plantilla puede incluir instrucciones, ejemplos de pocos disparos y un contexto y preguntas específicos apropiados para una tarea dada.

[LangChain](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/) proporciona herramientas para crear y trabajar con plantillas de prompts.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Responde a la pregunta basándote en el contexto a continuación. Si no puedes responder la pregunta, responde "No sé".

Contexto: {context}

Pregunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="LLM tiene un bonus a los trabajadores del 60% de su sueldo", question="¿Cuanto es el bonus en LLM?")

'Human: \nResponde a la pregunta basándote en el contexto a continuación. Si no puedes responder la pregunta, responde "No sé".\n\nContexto: LLM tiene un bonus a los trabajadores del 60% de su sueldo\n\nPregunta: ¿Cuanto es el bonus en LLM?\n'

## **Ahora podemos añadir a la cadena un nuevo componente el prompt**
prompt + model + parser

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "context": "LLM tiene un bonus a los trabajadores del 60% de su sueldo",
    "question": "¿Cuanto es el bonus en LLM?"
})

'El bonus en LLM sería del 60% del sueldo de los trabajadores.'

## **Combinar chains**

Podemos combinar diferentes cadenas para crear flujos de trabajo más complejos. Por ejemplo, creemos una segunda cadena que traduzca la respuesta de la primera cadena a un idioma diferente.




In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "Traduce {answer} al {language}"
)

In [ ]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "LLM tiene un bonus a los trabajadores del 60% de su sueldo",
        "question": "¿Cuanto es el bonus en LLM?",
        "language": "English",
    }
)

'The bonus in LLM is 60% of the salary.'

Cargamos el pdf o txt en memoria

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader


#loader = TextLoader("/data/v1/llm.txt")
loader = PyPDFLoader("llm.pdf");
text_documents = loader.load()
text_documents

[Document(page_content='LLM Sociedad Anonima\u2029', metadata={'source': 'llm.pdf', 'page': 0}),
 Document(page_content='Beneﬁcios al Empleado.Seguro de Salud Integral: Cobertura completa para el empleado y su familia, incluyendo consultas médicas, tratamientos especializados y hospitalización y dentista. Programa de Bienestar Mental: Acceso a terapias psicológicas y programas de manejo del estrés para promover la salud mental y el bienestar emocional.Horario Flexible: Posibilidad de ajustar los horarios de entrada y salida, y la opción de trabajo remoto para equilibrar las responsabilidades laborales y personales.Boniﬁcaciones Anuales: Bonos de rendimiento basados en los resultados personales y los de la empresa, pagaderos al ﬁnal de cada año ﬁscal. Bonus máximo del 60% de tu salario. Capacitación y Desarrollo: Programas de capacitación profesional y acceso a cursos para el desarrollo de habilidades personales y profesionales.Licencia Parental Extendida: Licencias por maternidad y pat

Existen numerosas maneras de fragmentar un documento. En este caso, emplearemos un divisor básico que segmenta el documento en partes de tamaño constante.

[Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
documents = text_splitter.split_documents(text_documents)
documents

[Document(page_content='LLM Sociedad Anonima', metadata={'source': 'llm.pdf', 'page': 0}),
 Document(page_content='Beneﬁcios al Empleado.Seguro de Salud Integral: Cobertura completa para el empleado y su familia, incluyendo consultas médicas, tratamientos especializados y hospitalización y dentista. Programa de Bienestar Mental: Acceso a terapias psicológicas y', metadata={'source': 'llm.pdf', 'page': 1}),
 Document(page_content='Mental: Acceso a terapias psicológicas y programas de manejo del estrés para promover la salud mental y el bienestar emocional.Horario Flexible: Posibilidad de ajustar los horarios de entrada y salida, y la opción de trabajo remoto para equilibrar', metadata={'source': 'llm.pdf', 'page': 1}),
 Document(page_content='y la opción de trabajo remoto para equilibrar las responsabilidades laborales y personales.Boniﬁcaciones Anuales: Bonos de rendimiento basados en los resultados personales y los de la empresa, pagaderos al ﬁnal de cada año ﬁscal. Bonus máximo del',

## **Encontrar los fragmentos relevantes a la pregunta realizada**

Ante una pregunta concreta, es necesario identificar los segmentos pertinentes de la transcripción para enviarlos al modelo. Esto es donde se destacan los embeddings.

Un embedding es una representación matemática del significado de una palabra, oración o documento, conceptualizado como una proyección en un espacio multidimensional. Una de las propiedades fundamentales de los embeddings es que los conceptos similares se ubican próximos entre sí en este espacio, mientras que los conceptos distintos se encuentran más separados. Para visualizar estos embeddings en dos dimensiones, puedes utilizar el "Embed Playground" de [Cohere](https://dashboard.cohere.com/playground/embed?redirect_uri=https%3A%2F%2Fdashboard.cohere.com%2Fplayground%2Fembed).

---




In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Cual es el bonus de LLM?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.013412422448485475, 0.002432951059624371, -0.0033392426437839954, -0.03895494056235842, -0.052429746717608014, -0.0005059983443141345, -0.036210072365897204, 0.013523326195184463, -0.02808637338585756, -0.028890426946408983]


Ahora vamos a comparar 2 textos, con la pregunta "Cual es el bonus de LLM?"

Para ellos vamos a utilizar la funcion Coseno, que calcula la relación entro la preguntas y los 2 textos.

In [ ]:
sentence1 = embeddings.embed_query("LLM tiene un bonus de 60%")
sentence2 = embeddings.embed_query("Me han robado la tarjeta, que hago")

from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9059610278278115, 0.7358235998226559)

Vector Store

Requerimos un método eficaz para almacenar segmentos de documentos, sus embeddings y efectuar búsquedas de similitud en gran escala. Para lograrlo, emplearemos un Vector Store.

Un Vector Store es una base de datos diseñada específicamente para embeddings, que facilita las búsquedas de similitud de manera rápida.

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstorelocal = DocArrayInMemorySearch.from_texts(
    [
        "LLM tiene un bonus de 60%",
        "LLM ofrece la posibilidad de teletrabajar",
        "podría teletrabajar en vuestra empresa?",
        "Me han robado la tarjeta, que hago",
        "Si te han robado la tarjeta de credito, llama al 900",
    ],
    embedding=embeddings,
)

vectorstorelocal.similarity_search_with_score(query="Cual es el bonus de LLM?", k=5)


[(Document(page_content='LLM tiene un bonus de 60%'), 0.9059610372317081),
 (Document(page_content='LLM ofrece la posibilidad de teletrabajar'),
  0.8401575278704726),
 (Document(page_content='podría teletrabajar en vuestra empresa?'),
  0.7717414550615617),
 (Document(page_content='Me han robado la tarjeta, que hago'),
  0.7358236262402932),
 (Document(page_content='Si te han robado la tarjeta de credito, llama al 900'),
  0.735286457893237)]

## **Conectamos la Vector Store a la Chain**

Es posible utilizar la Vector Store para identificar los segmentos más pertinentes de nuestro Texto en PDF que enviaremos al modelo. A continuación se describe cómo integrar La Vector Store en la cadena:

Primero, debemos establecer un Recuperador. Este Recuperador realizará una búsqueda por similitud dentro del La Vector Store y retornará los documentos más parecidos al próximo eslabón en la cadena.

In [ ]:
retriever1 = vectorstorelocal.as_retriever()
retriever1.invoke("Cual es el bonus de LLM?")

[Document(page_content='LLM tiene un bonus de 60%'),
 Document(page_content='LLM ofrece la posibilidad de teletrabajar'),
 Document(page_content='podría teletrabajar en vuestra empresa?'),
 Document(page_content='Me han robado la tarjeta, que hago')]

Nuestro prompt requiere dos parámetros: "contexto" y "pregunta". Con el uso del recuperador, podemos seleccionar los fragmentos que servirán de contexto para dar respuesta a la pregunta.

Es posible generar un mapa con estos dos elementos empleando las clases RunnableParallel y RunnablePassthrough. De esta forma, podemos entregar el contexto y la pregunta al prompt en forma de un mapa, identificando cada elemento con las claves "contexto" y "pregunta".

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Cual es el bonus de LLM?")

{'context': [Document(page_content='LLM tiene un bonus de 60%'),
  Document(page_content='LLM ofrece la posibilidad de teletrabajar'),
  Document(page_content='podría teletrabajar en vuestra empresa?'),
  Document(page_content='Me han robado la tarjeta, que hago')],
 'question': 'Cual es el bonus de LLM?'}

Lo añadimos al chain

In [ ]:
chain = setup | prompt | model | parser
chain.invoke("Cual es el bonus de LLM?")

'El bonus de LLM es del 60%.'

In [ ]:
chain.invoke("ofrece teletrabajo? o tengo que trabajar en la oficina")


'Sí, LLM ofrece la posibilidad de teletrabajar.'

## **Cargamos todo nuestro .pdf a la Vector Store**

In [ ]:
len(documents)

51

In [ ]:
vectorstorepdf = DocArrayInMemorySearch.from_documents(documents, embeddings)


In [ ]:
setuppdf = RunnableParallel(context=vectorstorepdf.as_retriever(), question=RunnablePassthrough())

chain = (
    setuppdf
    | prompt
    | model
    | parser
)
chain.invoke("que tiempo hace en Madrid")

'No sé.'

Pinecone como nuestra Vector Store

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = "llm-rag-curso"
pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

In [ ]:
pinecone.similarity_search("Puedo teletrabajar")[:3]


[Document(page_content='opciones de trabajo remoto para adaptarnos a las necesidades de nuestros empleados y clientes.¿Qué tipo de beneﬁcios ofrece la empresa?Ofrecemos varios beneﬁcios, incluyendo seguro médico, programas de bienestar, opciones de trabajo ﬂexible y', metadata={'page': 3.0, 'source': 'llm.pdf'}),
 Document(page_content='opciones de trabajo remoto para adaptarnos a las necesidades de nuestros empleados y clientes.¿Qué tipo de beneﬁcios ofrece la empresa?Ofrecemos varios beneﬁcios, incluyendo seguro médico, programas de bienestar, opciones de trabajo ﬂexible y', metadata={'page': 3.0, 'source': 'llm.pdf'}),
 Document(page_content='Mental: Acceso a terapias psicológicas y programas de manejo del estrés para promover la salud mental y el bienestar emocional.Horario Flexible: Posibilidad de ajustar los horarios de entrada y salida, y la opción de trabajo remoto para equilibrar', metadata={'page': 1.0, 'source': 'llm.pdf'})]

In [ ]:
setuppinecone = RunnableParallel(context=pinecone.as_retriever(), question=RunnablePassthrough())

chain = (
    setuppinecone
    | prompt
    | model
    | parser
)
chain.invoke("tiempo en madrid hoy")

'No sé.'